In [1]:
import pandas as pd
import geopandas as geo
import numpy as np
import matplotlib.pyplot as plt

In [40]:
 #Festnetz_gov nimmt maximum gerundet über alle verfügbaren Techniken/Anbieter

In [41]:
Festnetz = geo.read_file("AUTmap_Merged_festnetz.gpkg")

In [42]:
Festnetz_gov = geo.read_file("festnetz_gov.gpkg")

In [43]:
Festnetz = Festnetz.drop_duplicates("ID")

In [44]:
Festnetz_gov["Download_Max"] = Festnetz_gov.groupby("ID")["download"].transform("max")

In [45]:
Festnetz_gov = Festnetz_gov.drop_duplicates("ID")

In [46]:
Types_mobile =["LTE", "HSPA+","CELLULAR_ANY"]
Types =["WLAN","LAN","LTE","HSPA+","CELLULAR_ANY"]
Types_festnetz = ["WLAN","LAN"]

In [47]:
Festnetz_gov_merged = pd.merge(Festnetz_gov,Festnetz,on="ID", how="left")

In [48]:
Festnetz_gov_merged["Download_Mean"] = Festnetz_gov_merged["Download_Mean"].fillna(0)

In [49]:
Festnetz_gov_merged["Download_Max_y"] = Festnetz_gov_merged["Download_Max_y"].fillna(0)

In [50]:
Festnetz_gov_merged["Daten_wurden_verglichen"] = 0

In [51]:
Festnetz_gov_merged["relativerFehler_Mean"] = (1000*Festnetz_gov_merged["download"] -Festnetz_gov_merged["Download_Mean"] )/(1000*Festnetz_gov_merged["download"])

In [52]:
Festnetz_gov_merged["relativerFehler_Max"] = (1000*Festnetz_gov_merged["download"] -Festnetz_gov_merged["Download_Max_y"] )/(1000*Festnetz_gov_merged["download"])

In [53]:
Festnetz_gov_merged.loc[(Festnetz_gov_merged['download'] > 0)
                        &(Festnetz_gov_merged['Download_Mean'] > 0), 'Daten_wurden_verglichen'] = 1

In [54]:
#Festnetz_gov_merged_mitVergleichen = Festnetz_gov_merged[Festnetz_gov_merged["Daten_wurden_verglichen"] != 0]

In [55]:
#count = column[column > limit].count()

In [56]:
#Festnetz_gov_merged_mitVergleichen.relativerFehler_Mean[Festnetz_gov_merged_mitVergleichen.relativerFehler_Mean > -5].count()

In [57]:
#Festnetz_gov_merged_mitVergleichen_Mean = Festnetz_gov_merged_mitVergleichen[Festnetz_gov_merged["relativerFehler_Mean"] > -5]

In [58]:
#Festnetz_gov_merged_mitVergleichen_Max = Festnetz_gov_merged_mitVergleichen[Festnetz_gov_merged["relativerFehler_Max"] > -5]

In [59]:
#gdf1 = geo.GeoDataFrame(
 #   Festnetz_gov_merged)


In [60]:
gdf2 = geo.GeoDataFrame(
    Festnetz_gov_merged, geometry = Festnetz_gov_merged.geometry_x )


In [61]:
gdf2.columns

Index(['ID', 'NAME_x', 'l000100v3', 'infrastrukturanbieterin', 'technik',
       'download', 'upload', 'bearbeitung_bbb', 'geometry_x', 'Download_Max_x',
       'NAME_y', 'index_right', 'network_type', 'download_kbit', 'upload_kbit',
       'Download_Mean', 'Download_Max_y', 'Download_Median', 'Download_Count',
       'geometry_y', 'Daten_wurden_verglichen', 'relativerFehler_Mean',
       'relativerFehler_Max'],
      dtype='object')

In [ ]:
del gdf2["geometry_x"]

In [62]:
del gdf2["geometry_y"]

In [63]:
gdf2.to_file('festnetz_gov_relERROR_Mean.gpkg', driver='GPKG')  

C:\Users\Paul\anaconda3\envs\NewForGeopandas\lib\site-packages\geopandas\io\file.py:299: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  pd.Int64Index,


In [64]:
#gdf2.to_file('festnetz_gov_relERROR_Max.gpkg', driver='GPKG', layer='Festnetz_Gov')  

In [65]:
#gdf2 = geo.read_file("festnetz_gov_relERROR_Max.gpkg")

In [66]:
gdf2.columns

Index(['ID', 'NAME_x', 'l000100v3', 'infrastrukturanbieterin', 'technik',
       'download', 'upload', 'bearbeitung_bbb', 'geometry_x', 'Download_Max_x',
       'NAME_y', 'index_right', 'network_type', 'download_kbit', 'upload_kbit',
       'Download_Mean', 'Download_Max_y', 'Download_Median', 'Download_Count',
       'Daten_wurden_verglichen', 'relativerFehler_Mean',
       'relativerFehler_Max'],
      dtype='object')

In [81]:
gdf2[ 'download'] = gdf2[ 'download'] * 1000

In [97]:
bins = pd.IntervalIndex.from_tuples([(0, 10000), (10000, 30000), (30000, 100000), (100000, 1000000), (1000000, 10000000)],  closed='left')
labels=[1, 2, 3, 4, 5]
gdf2['download_gov_class'] = pd.cut(gdf2['download'], bins, labels=labels)

In [98]:
gdf2['download_rtr_class'] = pd.cut(gdf2['Download_Mean'], bins, labels=labels)

In [93]:
gdf2['download_gov_class'].value_counts()

[100000, 1000000)      201234
[10000, 30000)         172219
[30000, 100000)        170917
[0, 10000)              74546
[1000000, 10000000)     52282
Name: download_gov_class, dtype: int64

In [99]:
gdf2['download_rtr_class'].value_counts()

[0, 10000)             522370
[10000, 30000)          85882
[30000, 100000)         56816
[100000, 1000000)        6519
[1000000, 10000000)         2
Name: download_rtr_class, dtype: int64

In [105]:
gdf2['download_rtr_class'].cat.codes

0         0
1         0
2         0
3         0
4         0
         ..
671584    0
671585    0
671586    0
671587    0
671588    0
Length: 671589, dtype: int8

In [106]:
from sklearn.preprocessing import LabelEncoder

LE = LabelEncoder()


In [113]:
gdf2['download_class_dif'] = LE.fit_transform(gdf2['download_rtr_class']) - LE.fit_transform(gdf2['download_gov_class'])


In [118]:
gdf2['download_class_dif'].value_counts()

-1    198210
-2    177443
-3    148322
 0     97002
-4     32814
 1     14117
 2      3193
-5       301
 3       187
Name: download_class_dif, dtype: int64

In [121]:
gdf2.drop(['download_rtr_class','download_gov_class'], axis = 1).to_file('festnetz_gov_relERROR_Mean.gpkg', driver='GPKG')  

C:\Users\Paul\anaconda3\envs\NewForGeopandas\lib\site-packages\geopandas\io\file.py:299: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  pd.Int64Index,


In [130]:
masten = geo.read_file("Mobilfunksender/opencellid.gpkg", crs = "EPSG:4326")

In [132]:
masten = geo.GeoDataFrame(
    masten, geometry=geo.points_from_xy(masten.lon,masten.lat),crs="EPSG:4326")

In [127]:
masten = masten.drop_duplicates("cell")

In [128]:
masten.to_file('Mobilfunksender/Masten_geomertry.gpkg', driver='GPKG')  

C:\Users\Paul\anaconda3\envs\NewForGeopandas\lib\site-packages\geopandas\io\file.py:299: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  pd.Int64Index,


In [129]:
len(masten)


238505

In [2]:
Festnetz = geo.read_file("festnetz_gov_relERROR_Mean.gpkg")

In [3]:
Festnetz.columns

Index(['ID', 'NAME_x', 'l000100v3', 'infrastrukturanbieterin', 'technik',
       'download', 'upload', 'bearbeitung_bbb', 'Download_Max_x', 'NAME_y',
       'index_right', 'network_type', 'download_kbit', 'upload_kbit',
       'Download_Mean', 'Download_Max_y', 'Download_Median', 'Download_Count',
       'Daten_wurden_verglichen', 'relativerFehler_Mean',
       'relativerFehler_Max', 'download_class_dif', 'geometry'],
      dtype='object')

In [5]:
Festnetz = Festnetz[(Festnetz["Download_Mean"] != 0) & (Festnetz["download"] != 0)]

In [7]:
Festnetz.to_file('festnetz_gov_relERROR_Mean_cleaned.gpkg', driver='GPKG')

C:\Users\Paul\anaconda3\envs\NewForGeopandas\lib\site-packages\geopandas\io\file.py:299: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  pd.Int64Index,


In [15]:
Festnetz['download_class_dif'].value_counts()/len(Festnetz)*100

-1    31.811921
-2    27.883868
 0    18.474014
-3    10.938413
 1     7.566151
 2     1.711321
-4     1.505512
 3     0.100225
-5     0.008575
Name: download_class_dif, dtype: float64

In [16]:
Festnetz['download_class_dif'].value_counts()

-1    59355
-2    52026
 0    34469
-3    20409
 1    14117
 2     3193
-4     2809
 3      187
-5       16
Name: download_class_dif, dtype: int64